# Payoff Matrices (part 3)

> This module contains payoff matrices for different evolutionary games
>
> Part 3 contains payoff matrices for the following games
> - Regulatory Markets

In [1]:
# | default_exp `dummy_exp`


In [2]:
# | hide
# We could like to export all payoff matrices to the same module, `payoffs`.
# However,  we must set a default_exp and if we use the same module as
# another notebook, we will overwrite all existing exports with those in
# this file. Hence we set a dummy default_exp, `dummy_exp`, and each time
# we export a cell, we specify the `payoffs` module.


In [3]:
# | hide
# | export
from nbdev.showdoc import *
from fastcore.test import test_eq, test_close
import collections
import functools
from gh_pages_example.utils import *
from gh_pages_example.types import *
from gh_pages_example.methods import *
from gh_pages_example.model_utils import *
from gh_pages_example.payoffs import *
import itertools
import math
import typing

import fastcore.test
import more_itertools
import numpy as np
import nptyping


/mnt/d/Paolo-Work/git/gh-pages-example/gh_pages_example/model_utils.py:289: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if (ind not in allowed_inds) and (str(ind) not in allowed_inds):
/mnt/d/Paolo-Work/git/gh-pages-example/gh_pages_example/methods.py:189: ComplexWarning: Casting complex values to real discards the imaginary part
  ergodic = np.array(V.transpose(0, 2, 1)[y], dtype=float)


In [4]:
np.set_printoptions(suppress=True)  # don't use scientific notation


In [5]:
# | export payoffs
@method(build_payoffs, "regulatory_markets_v1_reward_before")
def build_payoffs(models):
    """Regulatory market payoffs when incentives are given in advance and only
    taken away if firms act unsafely."""
    names1 = ['b', 'c', 's', 'p', 'B', 'W']
    names2 = ['pfo_l', 'pfo_h', 'λ', 'r_l', 'r_h', 'g']
    b, c, s, p, B, W = [models[k] for k in names1]
    pfo_l, pfo_h, λ, r_l, r_h, g = [models[k] for k in names2]
    collective_risk = models.get('collective_risk', 0)
    risk_shared = (1 - (1-p)*collective_risk)
    
    Π_h11 = B / (2*W) + b/2 - c
    Π_h12 = b / (s+1) * (1 - pfo_h) * risk_shared + (b + B / W) * pfo_h - c
    Π_h21 = p * ( 1 - pfo_h) * (s*b / (s + 1) + s * B / W)
    Π_h22 = p * ( 1 - pfo_h**2) * (b/2 + s*B/(2*W)) * risk_shared
    
    Π_l11 = B / (2*W) + b/2 - c
    Π_l12 = b / (s+1) * (1 - pfo_l) * risk_shared + (b + B / W) * pfo_l - c
    Π_l21 = p * ( 1 - pfo_l) * (s*b / (s + 1) + s * B / W)
    Π_l22 = p * ( 1 - pfo_l**2) * (b/2 + s*B/(2*W)) * risk_shared
    
    λ_h = λ * (1 - p) * (1 - pfo_h)
    λ_l = λ * (1 - p) * (1 - pfo_l)
    
    Ω_11 = r_h + g
    Ω_12 = r_l + g
    Ω_21 = r_h + g * pfo_h**2 - λ_h
    Ω_22 = r_l + g * pfo_l**2 - λ_l
    Ω_31 = r_h + g
    Ω_32 = r_l + g * pfo_l**2 - λ_l
    
    payoffs = {}
    payoffs["4-1-1"] = {"P3": Ω_11,
                        "P2": Π_h11,
                        "P1": Π_h11}
    payoffs["4-1-2"] = {"P3": (Ω_11 + Ω_21) / 2,
                        "P2": Π_h12,
                        "P1": Π_h21}
    payoffs["4-1-3"] = {"P3": Ω_11,
                        "P2": Π_h11,
                        "P1": Π_h11}
    payoffs["4-2-1"] = {"P3": (Ω_11 + Ω_21) / 2,
                        "P2": Π_h21,
                        "P1": Π_h12}
    payoffs["4-2-2"] = {"P3": Ω_21,
                        "P2": Π_h22,
                        "P1": Π_h22}
    payoffs["4-2-3"] = {"P3": (Ω_11 + Ω_21) / 2,
                        "P2": Π_h21,
                        "P1": Π_h12}
    payoffs["4-3-1"] = {"P3": Ω_11,
                        "P2": Π_h11,
                        "P1": Π_h11}
    payoffs["4-3-2"] = {"P3": (Ω_11 + Ω_21) / 2,
                        "P2": Π_h12,
                        "P1": Π_h21}
    payoffs["4-3-3"] = {"P3": Ω_31,
                        "P2": Π_h11,
                        "P1": Π_h11}
    
    payoffs["5-1-1"] = {"P3": Ω_12,
                        "P2": Π_l11,
                        "P1": Π_l11}
    payoffs["5-1-2"] = {"P3": (Ω_12 + Ω_22) / 2,
                        "P2": Π_l12,
                        "P1": Π_l21}
    payoffs["5-1-3"] = {"P3": (Ω_12 + Ω_22) / 2,
                        "P2": Π_l12,
                        "P1": Π_l21}
    payoffs["5-2-1"] = {"P3": (Ω_12 + Ω_22) / 2,
                        "P2": Π_l21,
                        "P1": Π_l12}
    payoffs["5-2-2"] = {"P3": Ω_22,
                        "P2": Π_l22,
                        "P1": Π_l22}
    payoffs["5-2-3"] = {"P3": Ω_22,
                        "P2": Π_l22,
                        "P1": Π_l22}
    payoffs["5-3-1"] = {"P3": (Ω_12 + Ω_22) / 2,
                        "P2": Π_l21,
                        "P1": Π_l12}
    payoffs["5-3-2"] = {"P3": Ω_22,
                        "P2": Π_l22,
                        "P1": Π_l22}
    payoffs["5-3-3"] = {"P3": Ω_32,
                        "P2": Π_l22,
                        "P1": Π_l22}

    return {**models, "payoffs": payoffs}


In [6]:
# | export payoffs
@method(build_payoffs, "regulatory_markets_v1_reward_after")
def build_payoffs(models):
    """Regulatory market payoffs when there is only a reward after catching
    unsafe firms."""
    names1 = ['b', 'c', 's', 'p', 'B', 'W']
    names2 = ['pfo_l', 'pfo_h', 'λ', 'r_l', 'r_h', 'g']
    b, c, s, p, B, W = [models[k] for k in names1]
    pfo_l, pfo_h, λ, r_l, r_h, g = [models[k] for k in names2]
    collective_risk = models.get('collective_risk', 0)
    risk_shared = (1 - (1-p)*collective_risk)
    
    Π_h11 = B / (2*W) + b/2 - c
    Π_h12 = b / (s+1) * (1 - pfo_h) * risk_shared + (b + B / W) * pfo_h - c
    Π_h21 = p * ( 1 - pfo_h) * (s*b / (s + 1) + s * B / W)
    Π_h22 = p * ( 1 - pfo_h**2) * (b/2 + s*B/(2*W)) * risk_shared
    
    Π_l11 = B / (2*W) + b/2 - c
    Π_l12 = b / (s+1) * (1 - pfo_l) * risk_shared + (b + B / W) * pfo_l - c
    Π_l21 = p * ( 1 - pfo_l) * (s*b / (s + 1) + s * B / W)
    Π_l22 = p * ( 1 - pfo_l**2) * (b/2 + s*B/(2*W)) * risk_shared
    
    λ_h = λ * (1 - p) * (1 - pfo_h)
    λ_l = λ * (1 - p) * (1 - pfo_l)
    
    # No ex-ante reward for regulators
    Ω_11 = r_h
    Ω_12 = r_l
    # Expect to catch n*p unsafe firms, where n=2 and p=pfo_h
    # They may expect to be penalised if a disaster occurs under their watch
    # but by default the penalty λ may be 0.
    Ω_21 = r_h + g * 2 * pfo_h - λ_h
    Ω_22 = r_l + g * 2 * pfo_l - λ_l
    Ω_31 = r_h
    Ω_32 = r_l + g * 2 * pfo_l - λ_l
    
    payoffs = {}
    payoffs["4-1-1"] = {"P3": Ω_11,
                        "P2": Π_h11,
                        "P1": Π_h11}
    payoffs["4-1-2"] = {"P3": (Ω_11 + Ω_21) / 2,
                        "P2": Π_h12,
                        "P1": Π_h21}
    payoffs["4-1-3"] = {"P3": Ω_11,
                        "P2": Π_h11,
                        "P1": Π_h11}
    payoffs["4-2-1"] = {"P3": (Ω_11 + Ω_21) / 2,
                        "P2": Π_h21,
                        "P1": Π_h12}
    payoffs["4-2-2"] = {"P3": Ω_21,
                        "P2": Π_h22,
                        "P1": Π_h22}
    payoffs["4-2-3"] = {"P3": (Ω_11 + Ω_21) / 2,
                        "P2": Π_h21,
                        "P1": Π_h12}
    payoffs["4-3-1"] = {"P3": Ω_11,
                        "P2": Π_h11,
                        "P1": Π_h11}
    payoffs["4-3-2"] = {"P3": (Ω_11 + Ω_21) / 2,
                        "P2": Π_h12,
                        "P1": Π_h21}
    payoffs["4-3-3"] = {"P3": Ω_31,
                        "P2": Π_h11,
                        "P1": Π_h11}
    
    payoffs["5-1-1"] = {"P3": Ω_12,
                        "P2": Π_l11,
                        "P1": Π_l11}
    payoffs["5-1-2"] = {"P3": (Ω_12 + Ω_22) / 2,
                        "P2": Π_l12,
                        "P1": Π_l21}
    payoffs["5-1-3"] = {"P3": (Ω_12 + Ω_22) / 2,
                        "P2": Π_l12,
                        "P1": Π_l21}
    payoffs["5-2-1"] = {"P3": (Ω_12 + Ω_22) / 2,
                        "P2": Π_l21,
                        "P1": Π_l12}
    payoffs["5-2-2"] = {"P3": Ω_22,
                        "P2": Π_l22,
                        "P1": Π_l22}
    payoffs["5-2-3"] = {"P3": Ω_22,
                        "P2": Π_l22,
                        "P1": Π_l22}
    payoffs["5-3-1"] = {"P3": (Ω_12 + Ω_22) / 2,
                        "P2": Π_l21,
                        "P1": Π_l12}
    payoffs["5-3-2"] = {"P3": Ω_22,
                        "P2": Π_l22,
                        "P1": Π_l22}
    payoffs["5-3-3"] = {"P3": Ω_32,
                        "P2": Π_l22,
                        "P1": Π_l22}

    return {**models, "payoffs": payoffs}


In [7]:
# | export payoffs
@method(build_payoffs, "regulatory_markets_v1a_reward_after")
def build_payoffs(models):
    """An alternative payoff scheme for regulatory markets which more closely
    matches the DSAIR model"""
    names1 = ['b', 'c', 's', 'p', 'B', 'W']
    names2 = ['pfo_l', 'pfo_h', 'λ', 'r_l', 'r_h', 'g']
    b, c, s, p, B, W = [models[k] for k in names1]
    pfo_l, pfo_h, λ, r_l, r_h, g = [models[k] for k in names2]
    collective_risk = models.get('collective_risk', 0)
    risk_shared = (1 - (1-p)*collective_risk)
    
    Π_h11 = B / (2*W) + b/2 - c
    Π_h12 = b / (s+1) * (1 - pfo_h) * risk_shared + (b + B / W) * pfo_h - c
    Π_h21 = p * (s*b / (s + 1)  * ( 1 - pfo_h) + s * B / W)
    Π_h22 = p * (b/2 * ( 1 - pfo_h**2) + s*B/(2*W)) * risk_shared
    
    Π_l11 = B / (2*W) + b/2 - c
    Π_l12 = b / (s+1) * (1 - pfo_l) * risk_shared + (b + B / W) * pfo_l - c
    Π_l21 = p * (s*b / (s + 1) * ( 1 - pfo_l) + s * B / W)
    Π_l22 = p * (b/2 * ( 1 - pfo_l**2) + s*B/(2*W)) * risk_shared
    
    print("payoffs updated 1")
    
    λ_h = λ * (1 - p) * (1 - pfo_h)
    λ_l = λ * (1 - p) * (1 - pfo_l)
    
    # No ex-ante reward for regulators
    Ω_11 = r_h
    Ω_12 = r_l
    # Expect to catch n*p unsafe firms, where n=2 and p=pfo_h
    # They may expect to be penalised if a disaster occurs under their watch
    # but by default the penalty λ may be 0.
    Ω_21 = r_h + g * 2 * pfo_h - λ_h
    Ω_22 = r_l + g * 2 * pfo_l - λ_l
    Ω_31 = r_h
    Ω_32 = r_l + g * 2 * pfo_l - λ_l
    
    payoffs = {}
    payoffs["4-1-1"] = {"P3": Ω_11,
                        "P2": Π_h11,
                        "P1": Π_h11}
    payoffs["4-1-2"] = {"P3": (Ω_11 + Ω_21) / 2,
                        "P2": Π_h12,
                        "P1": Π_h21}
    payoffs["4-1-3"] = {"P3": Ω_11,
                        "P2": Π_h11,
                        "P1": Π_h11}
    payoffs["4-2-1"] = {"P3": (Ω_11 + Ω_21) / 2,
                        "P2": Π_h21,
                        "P1": Π_h12}
    payoffs["4-2-2"] = {"P3": Ω_21,
                        "P2": Π_h22,
                        "P1": Π_h22}
    payoffs["4-2-3"] = {"P3": (Ω_11 + Ω_21) / 2,
                        "P2": Π_h21,
                        "P1": Π_h12}
    payoffs["4-3-1"] = {"P3": Ω_11,
                        "P2": Π_h11,
                        "P1": Π_h11}
    payoffs["4-3-2"] = {"P3": (Ω_11 + Ω_21) / 2,
                        "P2": Π_h12,
                        "P1": Π_h21}
    payoffs["4-3-3"] = {"P3": Ω_31,
                        "P2": Π_h11,
                        "P1": Π_h11}
    
    payoffs["5-1-1"] = {"P3": Ω_12,
                        "P2": Π_l11,
                        "P1": Π_l11}
    payoffs["5-1-2"] = {"P3": (Ω_12 + Ω_22) / 2,
                        "P2": Π_l12,
                        "P1": Π_l21}
    payoffs["5-1-3"] = {"P3": (Ω_12 + Ω_22) / 2,
                        "P2": Π_l12,
                        "P1": Π_l21}
    payoffs["5-2-1"] = {"P3": (Ω_12 + Ω_22) / 2,
                        "P2": Π_l21,
                        "P1": Π_l12}
    payoffs["5-2-2"] = {"P3": Ω_22,
                        "P2": Π_l22,
                        "P1": Π_l22}
    payoffs["5-2-3"] = {"P3": Ω_22,
                        "P2": Π_l22,
                        "P1": Π_l22}
    payoffs["5-3-1"] = {"P3": (Ω_12 + Ω_22) / 2,
                        "P2": Π_l21,
                        "P1": Π_l12}
    payoffs["5-3-2"] = {"P3": Ω_22,
                        "P2": Π_l22,
                        "P1": Π_l22}
    payoffs["5-3-3"] = {"P3": Ω_32,
                        "P2": Π_l22,
                        "P1": Π_l22}

    return {**models, "payoffs": payoffs}


In [8]:
# | hide
import nbdev
nbdev.nbdev_export()
